# Описание проекта

Компания собрала исторические данные о заказах такси в аэропортах. Чтобы привлекать больше водителей в период пиковой нагрузки, нужно спрогнозировать количество заказов такси на следующий час. Необходимо построить модель для такого предсказания.
В качестве метрики используем RMSE. Удовлетворительным уровнем будет считаться, если метрика на тестовой выборке не будет превышать 48.

Порядок выполнения проекта:
1. Загрузим и подготовим данные, выполнив ресемплирование по одному часу.
2. Проанализируем данные
3. Обучим 2 модели: линейную регрессию и градиентный бустинг. Размер тестовой выборки составит 10% от всех данных.
4. Проверим качество моделей на тестовой выборке.
5. Сделаем выводы по проделанной работе.

## Описание данных

In [34]:
#import
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')

## Загрузка данных:

Загрузим данные, после чего проверим корректность типов данных и наличие пропусков. Ресемплируем данные по одному часу, чтобы получить количество заказов такси в каждом часе.

In [35]:
# Загружаем данные
try:
    df = pd.read_csv('datasets/taxi.csv', parse_dates=['datetime'])
except FileNotFoundError:
    df = pd.read_csv('data/taxi.csv', parse_dates=['datetime'])

In [36]:
# Проверяем порядок и период дат
print(f'Первая строка датасета:             {df.loc[df.index[0], 'datetime']}')
print(f'Самая ранняя запись в датасете:     {df["datetime"].min()}\n\n')

print(f'Последняя строка датасета:          {df.loc[df.index[-1], 'datetime']}')
print(f'Самая поздняя запись в датасете:    {df["datetime"].max()}\n\n')

# Выводим датасет
df

Первая строка датасета:             2018-03-01 00:00:00
Самая ранняя запись в датасете:     2018-03-01 00:00:00


Последняя строка датасета:          2018-08-31 23:50:00
Самая поздняя запись в датасете:    2018-08-31 23:50:00




,datetime,num_orders
0,2018-03-01 00:00:00,9
1,2018-03-01 00:10:00,14
2,2018-03-01 00:20:00,28
3,2018-03-01 00:30:00,20
4,2018-03-01 00:40:00,32
...,...,...
26491,2018-08-31 23:10:00,32
26492,2018-08-31 23:20:00,24
26493,2018-08-31 23:30:00,27
26494,2018-08-31 23:40:00,39


In [37]:
 # Выводим информацию о датасете
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26496 entries, 0 to 26495
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   datetime    26496 non-null  datetime64[ns]
 1   num_orders  26496 non-null  int64         
dtypes: datetime64[ns](1), int64(1)
memory usage: 414.1 KB


In [38]:
# Ресемплируем данные
df = df.resample('1H', on='datetime').sum()
df

,num_orders
datetime,
2018-03-01 00:00:00,124
2018-03-01 01:00:00,85
2018-03-01 02:00:00,71
2018-03-01 03:00:00,66
2018-03-01 04:00:00,43
...,...
2018-08-31 19:00:00,136
2018-08-31 20:00:00,154
2018-08-31 21:00:00,159


In [39]:
# Проверяем корректность
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 4416 entries, 2018-03-01 00:00:00 to 2018-08-31 23:00:00
Freq: h
Data columns (total 1 columns):
 #   Column      Non-Null Count  Dtype
---  ------      --------------  -----
 0   num_orders  4416 non-null   int64
dtypes: int64(1)
memory usage: 69.0 KB


### Выводы:
- Данные загружены корректно, типы данных соответствуют ожидаемым.
- Пропусков в данных нет.
- Данные ресемплированы по одному часу, общее количество записей сократилось с 26 500 до 4 500
- Столбец `datetime` стал индексом, что позволит удобно работать с временными рядами.